# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096891


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:50,  1.79s/it]

Rendering models:  10%|█         | 3/30 [00:04<00:44,  1.64s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:53,  2.06s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:39,  1.60s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:17<00:01,  3.05it/s]

Rendering models:  90%|█████████ | 27/30 [00:18<00:01,  2.11it/s]

Rendering models:  93%|█████████▎| 28/30 [00:19<00:00,  2.28it/s]

Rendering models:  97%|█████████▋| 29/30 [00:19<00:00,  2.18it/s]

not-logged-in-3b0b929f4475fd276561     0.002206
martianmax                             0.000258
not-logged-in-f737655ff84da201cfb7     0.004154
DevionJ                                0.017459
jnarayanbvg                            0.013971
not-logged-in-56800035d100de482bcc     0.000929
PRivate_pAuLa                          0.000564
not-logged-in-c6917d9b19f41e6c2995     0.004885
awright5                               0.001908
thiagoxxxx                             0.000345
SaffronGB                              0.001624
redboy21                               0.056952
Farquar                                0.000501
sn346808                               0.021620
ElisabethB                             0.000218
KrisKeogh                              0.004834
caseyroberto                           0.000297
not-logged-in-8262566d1147ff08203a    11.508935
jamesxu                                0.001588
superphoton                            0.000240
puic                                   0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())